## Pattern 1: Producer-Consumer Pipeline

```
┌──────────────────────────────────────────────────────────────┐
│                   Producer-Consumer Pipeline                  │
├──────────────────────────────────────────────────────────────┤
│                                                              │
│   ┌─────────┐      ┌─────────────┐      ┌──────────┐        │
│   │Producer │─────▶│ GPU Memory  │─────▶│ Consumer │        │
│   │ Process │      │ (Shared IPC)│      │ Process  │        │
│   └─────────┘      └─────────────┘      └──────────┘        │
│        │                  │                   │              │
│        │           ┌──────┴──────┐           │              │
│        │           │ Ring Buffer │           │              │
│        │           │  N Slots    │           │              │
│        ▼           └─────────────┘           ▼              │
│   Write slot 0         ....            Read slot 0          │
│   Write slot 1                         Read slot 1          │
│      ...                                  ...               │
└──────────────────────────────────────────────────────────────┘
```

In [ ]:
%%writefile ipc_ring_buffer.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <sys/wait.h>
#include <unistd.h>
#include <sys/mman.h>
#include <atomic>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error: %s\n", cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

constexpr int NUM_SLOTS = 4;
constexpr int SLOT_SIZE = 1024 * 1024;  // 1M floats per slot
constexpr int NUM_ITERATIONS = 10;

__global__ void produceData(float* slot, int slotIdx, int iteration) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < SLOT_SIZE) {
        slot[idx] = (float)(iteration * 1000 + slotIdx * 100 + idx % 100);
    }
}

__global__ void consumeData(float* slot, int slotIdx, int iteration, float* result) {
    __shared__ float sdata[256];
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Compute sum of slot data (reduction)
    float val = (idx < SLOT_SIZE) ? slot[idx] : 0.0f;
    sdata[threadIdx.x] = val;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) sdata[threadIdx.x] += sdata[threadIdx.x + s];
        __syncthreads();
    }
    
    if (threadIdx.x == 0) atomicAdd(result, sdata[0]);
}

struct RingBuffer {
    cudaIpcMemHandle_t handles[NUM_SLOTS];
    std::atomic<int> writeIdx;  // Next slot to write
    std::atomic<int> readIdx;   // Next slot to read
    std::atomic<int> count;     // Number of filled slots
    std::atomic<bool> done;
};

int main() {
    printf("=== IPC Ring Buffer Demo ===\n");
    printf("Slots: %d, Size per slot: %d floats\n\n", NUM_SLOTS, SLOT_SIZE);
    
    // Create shared ring buffer metadata
    RingBuffer* ring = (RingBuffer*)mmap(NULL, sizeof(RingBuffer),
        PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);
    ring->writeIdx = 0;
    ring->readIdx = 0;
    ring->count = 0;
    ring->done = false;
    
    pid_t pid = fork();
    
    if (pid == 0) {
        // ========== CONSUMER PROCESS ==========
        usleep(100000);  // Let producer initialize
        
        // Open all slot handles
        float* d_slots[NUM_SLOTS];
        for (int i = 0; i < NUM_SLOTS; i++) {
            CHECK_CUDA(cudaIpcOpenMemHandle((void**)&d_slots[i], ring->handles[i],
                                             cudaIpcMemLazyEnablePeerAccess));
        }
        
        float* d_result;
        CHECK_CUDA(cudaMalloc(&d_result, sizeof(float)));
        
        int consumed = 0;
        while (consumed < NUM_ITERATIONS) {
            // Wait for data
            while (ring->count == 0 && !ring->done) {
                usleep(100);
            }
            
            if (ring->count > 0) {
                int slot = ring->readIdx % NUM_SLOTS;
                
                CHECK_CUDA(cudaMemset(d_result, 0, sizeof(float)));
                consumeData<<<(SLOT_SIZE+255)/256, 256>>>(d_slots[slot], slot, consumed, d_result);
                CHECK_CUDA(cudaDeviceSynchronize());
                
                float h_result;
                CHECK_CUDA(cudaMemcpy(&h_result, d_result, sizeof(float), cudaMemcpyDeviceToHost));
                printf("[Consumer] Iteration %d, Slot %d, Sum: %.2e\n", consumed, slot, h_result);
                
                ring->readIdx++;
                ring->count--;
                consumed++;
            }
        }
        
        // Cleanup
        for (int i = 0; i < NUM_SLOTS; i++) {
            CHECK_CUDA(cudaIpcCloseMemHandle(d_slots[i]));
        }
        CHECK_CUDA(cudaFree(d_result));
        
        printf("[Consumer] Done!\n");
        exit(0);
        
    } else {
        // ========== PRODUCER PROCESS ==========
        
        // Allocate ring buffer slots
        float* d_slots[NUM_SLOTS];
        for (int i = 0; i < NUM_SLOTS; i++) {
            CHECK_CUDA(cudaMalloc(&d_slots[i], SLOT_SIZE * sizeof(float)));
            CHECK_CUDA(cudaIpcGetMemHandle(&ring->handles[i], d_slots[i]));
        }
        printf("[Producer] Allocated %d slots\n", NUM_SLOTS);
        
        for (int iter = 0; iter < NUM_ITERATIONS; iter++) {
            // Wait for free slot
            while (ring->count >= NUM_SLOTS) {
                usleep(100);
            }
            
            int slot = ring->writeIdx % NUM_SLOTS;
            produceData<<<(SLOT_SIZE+255)/256, 256>>>(d_slots[slot], slot, iter);
            CHECK_CUDA(cudaDeviceSynchronize());
            
            printf("[Producer] Iteration %d, Slot %d filled\n", iter, slot);
            
            ring->writeIdx++;
            ring->count++;
        }
        
        ring->done = true;
        
        // Wait for consumer
        wait(NULL);
        
        // Cleanup
        for (int i = 0; i < NUM_SLOTS; i++) {
            CHECK_CUDA(cudaFree(d_slots[i]));
        }
        munmap(ring, sizeof(RingBuffer));
        
        printf("\n[Producer] Done! Pipeline complete.\n");
    }
    
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 -std=c++14 ipc_ring_buffer.cu -o ipc_ring_buffer && ./ipc_ring_buffer

## Pattern 2: Inference Server Architecture

```
┌────────────────────────────────────────────────────────────────┐
│                    Inference Server Pattern                     │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│                     ┌─────────────────┐                        │
│                     │  Model Server   │                        │
│                     │ (Holds Weights) │                        │
│                     └────────┬────────┘                        │
│                              │                                 │
│                    IPC Memory Handles                          │
│                              │                                 │
│         ┌────────────────────┼────────────────────┐            │
│         ▼                    ▼                    ▼            │
│   ┌──────────┐        ┌──────────┐        ┌──────────┐        │
│   │ Worker 1 │        │ Worker 2 │        │ Worker N │        │
│   │ (Batch A)│        │ (Batch B)│        │ (Batch N)│        │
│   └──────────┘        └──────────┘        └──────────┘        │
│                                                                │
│   Workers share model weights via IPC                          │
│   Each worker has own input/output buffers                     │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
%%writefile inference_server.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <sys/wait.h>
#include <unistd.h>
#include <sys/mman.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error: %s\n", cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

constexpr int MODEL_SIZE = 10 * 1024 * 1024;  // 10M parameters
constexpr int BATCH_SIZE = 1024;
constexpr int NUM_WORKERS = 3;

// Simple "inference" kernel - matrix-vector multiply
__global__ void inference(const float* weights, const float* input, 
                          float* output, int modelSize, int batchSize) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < batchSize) {
        float sum = 0.0f;
        // Each output is dot product with subset of weights
        int stride = modelSize / batchSize;
        for (int i = 0; i < stride; i++) {
            sum += weights[idx * stride + i] * input[idx];
        }
        output[idx] = tanhf(sum);  // Activation
    }
}

struct ServerState {
    cudaIpcMemHandle_t weightsHandle;
    volatile bool ready;
    volatile int workersFinished;
};

void runWorker(int workerId, ServerState* state) {
    printf("[Worker %d] Started (PID %d)\n", workerId, getpid());
    
    // Wait for server
    while (!state->ready) usleep(1000);
    
    // Open shared weights
    float* d_weights;
    CHECK_CUDA(cudaIpcOpenMemHandle((void**)&d_weights, state->weightsHandle,
                                     cudaIpcMemLazyEnablePeerAccess));
    printf("[Worker %d] Opened shared model weights\n", workerId);
    
    // Allocate local input/output
    float *d_input, *d_output;
    CHECK_CUDA(cudaMalloc(&d_input, BATCH_SIZE * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_output, BATCH_SIZE * sizeof(float)));
    
    // Prepare input (different for each worker)
    float* h_input = new float[BATCH_SIZE];
    for (int i = 0; i < BATCH_SIZE; i++) {
        h_input[i] = (float)(workerId + 1) * 0.1f;
    }
    CHECK_CUDA(cudaMemcpy(d_input, h_input, BATCH_SIZE * sizeof(float), cudaMemcpyHostToDevice));
    
    // Run inference
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));
    
    CHECK_CUDA(cudaEventRecord(start));
    for (int i = 0; i < 100; i++) {  // 100 inference runs
        inference<<<(BATCH_SIZE+255)/256, 256>>>(d_weights, d_input, d_output, MODEL_SIZE, BATCH_SIZE);
    }
    CHECK_CUDA(cudaEventRecord(stop));
    CHECK_CUDA(cudaEventSynchronize(stop));
    
    float ms;
    CHECK_CUDA(cudaEventElapsedTime(&ms, start, stop));
    printf("[Worker %d] 100 inferences in %.2f ms (%.2f ms/inference)\n", 
           workerId, ms, ms/100);
    
    // Get sample output
    float h_output[5];
    CHECK_CUDA(cudaMemcpy(h_output, d_output, 5*sizeof(float), cudaMemcpyDeviceToHost));
    printf("[Worker %d] Sample outputs: [%.4f, %.4f, %.4f, %.4f, %.4f]\n",
           workerId, h_output[0], h_output[1], h_output[2], h_output[3], h_output[4]);
    
    // Cleanup
    CHECK_CUDA(cudaIpcCloseMemHandle(d_weights));
    CHECK_CUDA(cudaFree(d_input));
    CHECK_CUDA(cudaFree(d_output));
    delete[] h_input;
    
    __sync_fetch_and_add(&state->workersFinished, 1);
    printf("[Worker %d] Done!\n", workerId);
}

int main() {
    printf("=== Inference Server Demo ===\n");
    printf("Model size: %d parameters (%.1f MB)\n", MODEL_SIZE, MODEL_SIZE * 4.0f / 1e6);
    printf("Workers: %d\n\n", NUM_WORKERS);
    
    ServerState* state = (ServerState*)mmap(NULL, sizeof(ServerState),
        PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);
    state->ready = false;
    state->workersFinished = 0;
    
    // Fork workers
    for (int w = 0; w < NUM_WORKERS; w++) {
        if (fork() == 0) {
            runWorker(w, state);
            exit(0);
        }
    }
    
    // ========== SERVER PROCESS ==========
    printf("[Server] Allocating model weights...\n");
    
    float* d_weights;
    CHECK_CUDA(cudaMalloc(&d_weights, MODEL_SIZE * sizeof(float)));
    
    // Initialize weights (normally would load from file)
    float* h_weights = new float[MODEL_SIZE];
    for (int i = 0; i < MODEL_SIZE; i++) {
        h_weights[i] = ((float)rand() / RAND_MAX - 0.5f) * 0.1f;
    }
    CHECK_CUDA(cudaMemcpy(d_weights, h_weights, MODEL_SIZE * sizeof(float), cudaMemcpyHostToDevice));
    printf("[Server] Model weights initialized\n");
    
    // Create IPC handle
    CHECK_CUDA(cudaIpcGetMemHandle(&state->weightsHandle, d_weights));
    printf("[Server] IPC handle created\n");
    
    // Signal workers
    state->ready = true;
    printf("[Server] Workers notified, waiting for completion...\n\n");
    
    // Wait for all workers
    while (state->workersFinished < NUM_WORKERS) {
        usleep(10000);
    }
    
    // Reap children
    for (int w = 0; w < NUM_WORKERS; w++) {
        wait(NULL);
    }
    
    // Cleanup
    CHECK_CUDA(cudaFree(d_weights));
    delete[] h_weights;
    munmap(state, sizeof(ServerState));
    
    printf("\n[Server] All workers completed. Server shutdown.\n");
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 inference_server.cu -o inference_server && ./inference_server

## Pattern 3: Multi-GPU IPC with Peer Access

When processes use different GPUs, we need peer access:

In [ ]:
%%writefile multi_gpu_ipc.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <sys/wait.h>
#include <unistd.h>
#include <sys/mman.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error: %s\n", cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

__global__ void writePattern(float* data, int n, float value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = value + idx;
}

__global__ void readAndVerify(float* data, int n, float expected, int* errors) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        if (fabsf(data[idx] - (expected + idx)) > 0.001f) {
            atomicAdd(errors, 1);
        }
    }
}

struct SharedState {
    cudaIpcMemHandle_t handle;
    volatile bool ready;
    volatile bool done;
};

int main() {
    int numDevices;
    CHECK_CUDA(cudaGetDeviceCount(&numDevices));
    printf("=== Multi-GPU IPC Demo ===\n");
    printf("Found %d GPU(s)\n\n", numDevices);
    
    if (numDevices < 2) {
        printf("Need at least 2 GPUs for this demo.\n");
        printf("On HPC: srun --partition=h100dualflex --gres=gpu:2 ...\n");
        return 1;
    }
    
    // Check peer access
    int canAccess01, canAccess10;
    CHECK_CUDA(cudaDeviceCanAccessPeer(&canAccess01, 0, 1));
    CHECK_CUDA(cudaDeviceCanAccessPeer(&canAccess10, 1, 0));
    printf("GPU 0 -> GPU 1 peer access: %s\n", canAccess01 ? "YES" : "NO");
    printf("GPU 1 -> GPU 0 peer access: %s\n\n", canAccess10 ? "YES" : "NO");
    
    SharedState* state = (SharedState*)mmap(NULL, sizeof(SharedState),
        PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);
    state->ready = false;
    state->done = false;
    
    const int N = 1024 * 1024;
    
    pid_t pid = fork();
    
    if (pid == 0) {
        // ========== PROCESS 2: GPU 1 ==========
        CHECK_CUDA(cudaSetDevice(1));
        printf("[Process 2] Using GPU 1\n");
        
        // Enable peer access
        if (canAccess10) {
            CHECK_CUDA(cudaDeviceEnablePeerAccess(0, 0));
            printf("[Process 2] Enabled peer access to GPU 0\n");
        }
        
        while (!state->ready) usleep(1000);
        
        // Open shared memory from GPU 0
        float* d_data;
        CHECK_CUDA(cudaIpcOpenMemHandle((void**)&d_data, state->handle,
                                         cudaIpcMemLazyEnablePeerAccess));
        printf("[Process 2] Opened GPU 0 memory\n");
        
        // Read and verify data written by GPU 0
        int* d_errors;
        CHECK_CUDA(cudaMalloc(&d_errors, sizeof(int)));
        CHECK_CUDA(cudaMemset(d_errors, 0, sizeof(int)));
        
        readAndVerify<<<(N+255)/256, 256>>>(d_data, N, 42.0f, d_errors);
        CHECK_CUDA(cudaDeviceSynchronize());
        
        int errors;
        CHECK_CUDA(cudaMemcpy(&errors, d_errors, sizeof(int), cudaMemcpyDeviceToHost));
        printf("[Process 2] Verification: %s (%d errors)\n", 
               errors == 0 ? "PASSED" : "FAILED", errors);
        
        CHECK_CUDA(cudaIpcCloseMemHandle(d_data));
        CHECK_CUDA(cudaFree(d_errors));
        state->done = true;
        
        printf("[Process 2] Done!\n");
        exit(0);
        
    } else {
        // ========== PROCESS 1: GPU 0 ==========
        CHECK_CUDA(cudaSetDevice(0));
        printf("[Process 1] Using GPU 0\n");
        
        // Allocate and initialize
        float* d_data;
        CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(float)));
        writePattern<<<(N+255)/256, 256>>>(d_data, N, 42.0f);
        CHECK_CUDA(cudaDeviceSynchronize());
        printf("[Process 1] Data initialized\n");
        
        // Share via IPC
        CHECK_CUDA(cudaIpcGetMemHandle(&state->handle, d_data));
        state->ready = true;
        printf("[Process 1] Handle shared, waiting...\n");
        
        while (!state->done) usleep(1000);
        
        wait(NULL);
        CHECK_CUDA(cudaFree(d_data));
        munmap(state, sizeof(SharedState));
        
        printf("\n=== Multi-GPU IPC Complete! ===\n");
    }
    
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 multi_gpu_ipc.cu -o multi_gpu_ipc
print("Run with: srun --partition=h100dualflex --gres=gpu:2 ./multi_gpu_ipc")

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice IPC patterns:

**Exercise 1: Enhanced Ring Buffer**
- Add support for variable-size messages
- Implement a priority queue variant

**Exercise 2: Multi-Consumer Pattern**
- Extend the producer-consumer to support multiple consumers
- Implement fair scheduling between consumers

**Exercise 3: Bidirectional IPC**
- Create a pattern where both processes can be producers and consumers
- Implement request-response communication

In [ ]:
%%writefile ipc_exercises.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <sys/wait.h>
#include <unistd.h>
#include <sys/mman.h>
#include <string.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error: %s\n", cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: Enhanced Ring Buffer with Variable-Size Messages
// TODO: Modify the ring buffer to support variable-size messages
// Hint: Use a header that stores message size before each message
// =============================================================================

struct MessageHeader {
    size_t size;      // Size of the following data
    int priority;     // For priority queue extension
};

// TODO: Implement variable-size ring buffer here
// - Each slot should contain a header + data
// - Consumer should read header first to know data size


// =============================================================================
// Exercise 2: Multi-Consumer Pattern
// TODO: Extend to support multiple consumers reading different slots
// Hint: Use atomic operations for slot claiming
// =============================================================================

struct MultiConsumerBuffer {
    int* data;
    int* read_index;   // Shared atomic read pointer
    int* write_index;  // Producer write pointer
    int capacity;
};

__device__ int atomicClaimSlot(int* read_index, int write_index, int capacity) {
    // TODO: Atomically claim a slot for reading
    // Return -1 if no data available
    int old = atomicAdd(read_index, 1);
    if (old < write_index) {
        return old % capacity;
    }
    atomicSub(read_index, 1);  // Undo if nothing to read
    return -1;
}


// =============================================================================
// Exercise 3: Bidirectional IPC (Request-Response)
// TODO: Implement a pattern where processes can send requests and receive responses
// =============================================================================

struct BidirectionalChannel {
    int* request_buffer;
    int* response_buffer;
    int* request_ready;   // Flag: 1 = request pending
    int* response_ready;  // Flag: 1 = response ready
};

// TODO: Implement send_request() and wait_for_response() functions
// TODO: Implement receive_request() and send_response() functions


// =============================================================================
// Main: Test your implementations
// =============================================================================
int main() {
    printf("=== IPC Pattern Exercises ===\n\n");
    
    // Exercise 1 placeholder
    printf("Exercise 1: Enhanced Ring Buffer\n");
    printf("TODO: Implement variable-size message support\n\n");
    
    // Exercise 2 placeholder
    printf("Exercise 2: Multi-Consumer Pattern\n");
    printf("TODO: Implement fair scheduling between consumers\n\n");
    
    // Exercise 3 placeholder
    printf("Exercise 3: Bidirectional IPC\n");
    printf("TODO: Implement request-response communication\n\n");
    
    printf("Hint: Fork processes and use IPC memory handles for testing\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o ipc_exercises ipc_exercises.cu && ./ipc_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises use CuPy for IPC operations (Numba doesn't directly support IPC):

1. **Process Communication with CuPy**: Use `cupy.cuda.runtime.ipcGetMemHandle()` to share memory between Python processes
2. **Multiprocessing with GPU**: Create a producer-consumer pattern using Python's `multiprocessing` module with CuPy arrays
3. **Performance Comparison**: Compare IPC overhead vs. copying data through CPU memory

*Note: Full IPC functionality requires the `multiprocessing` module and careful handle management.*

## Summary

| Pattern | Use Case | Key Benefit |
|---------|----------|-------------|
| Ring Buffer | Streaming data | Decouples producer/consumer speed |
| Inference Server | ML serving | Share large model weights |
| Multi-GPU IPC | HPC/MPI | Cross-GPU zero-copy access |

**Best Practices:**
1. Keep producer alive while consumers use handles
2. Use events for synchronization, not just polling
3. Enable peer access for cross-GPU scenarios
4. Consider NCCL for high-performance collective operations